# Lesson 11: Network Models

So far, all of our models have been of single neurons. Although we either implicitly or explicitly account for the fact that individual neurons receive inputs from many different sources, we have not considered the dynamics of the networks in which these neurons are embedded. These dynamics support many important neural computations.

As you can imagine, network models can have many, many state variables and parameters, leading to greatly increased complexity, and we only have time to survey some major types of network models. However, the lessons we can learn from simple linear and nonlinear systems will help to shed light on some of the principles of brain function.

### Readings

Many of the figures in this notebook are from Chapter 9 in *Principles of Computational Modelling in Neuroscience*, by David Sterratt et al (2011) and from Chapter 7 in *Theoretical Neuroscience* by Dayan and Abbott (2001). 

## Components of Neuronal Networks

Neuronal networks are highly diverse. They typically comprise multiple kinds of cells with distinct biophysical dynamics.

For example, here are some of firing patterns seen in GABAergic interneurons (Ascoli et al 2008):

<img src="images/l13_ascoli2008_firing-patterns.png" alt="Ascoli 2008 firing patterns" style="width: 400px;"/>

## Connectivity in Neuronal Networks 
Neurons in the CNS are highly interconnected. For example, a typical neocortical pyramidal cell receives about 10,000 synapses.

Connections can be electrical (gap junctions) or synaptic (transmitter release binding to receptors). 

Synaptic neurotransmitters can be excitatory, inhibitory, and modulatory, depending on what receptors the postsynaptic cell expresses.

Spatial distribution of connections is not uniform:

- long-range: afferent (input) and efferent (output)
    - primarily excitatory
    - topographically organized
- lateral (within a cortical area or nucleus)
    - primarily excitatory
    - in some systems, more likely between similarly tuned neurons
- local (within a column)
    - excitatory and inhibitory

## Modeling networks

The biggest consideration in building network models is how much realism to include.

- biophysical vs abstract dynamics
- real space vs abstract topography
- diverse dynamics or representative types
- continuous time, discrete time, or event-based

What are the costs and benefits of increased realism?

### Detailed models

A model striving for realism will include many of the following:

- conductance-based dynamics
- multiple compartments
- multiple cell types
- spatially organized interconnections

Some major drawbacks (Traub et al 2005):

- large numbers of parameters and limited data to constrain them
- expense and slowness of computations
- low explanatory power
- hard to compare predictions to experiment

Nonetheless, there have been a number of large-scale, realistic models constructed and studied extensively.

#### Traub et al (2005)

- single cortical column comprising 3360 neurons
- 200 thalamic relay and reticular formation cells
- 8 different cell types (all members of each type the same)
- "cartoon" morphology and connectivity
    
<img src="images/l13_traub2005_fig1.jpeg" alt="Traub et al (2005) morphologies" style="width: 400px;"/>
    

#### Markram et al (2006) "Blue Brain":
- single cortical column comprising 31,000 neurons
- 207 different cell types, with diverse members
- 37 million synapses, based on detailed morphology

<img src="images/l13_blue_brain.png" alt="Blue Brain Project" style="width: 500px;"/>


### Simplified and schematic models

Obviously, these kinds of simulations are far beyond the scope of our class.

We could try to simplify the realistic models by removing details:

- replace conductance-based neurons with GLM, LIF, or simplified spiking models (e.g. QIF)
- remove spatial organization of connectivity
- scale down the number of neurons and synapses (while maintaining proportions and connectivity)

Or, we can take inspiration from the idea of topological equivalence, and construct abstract models that represent different *kinds* of network structure.

### Network topology

- Density of connections
    - probability that neurons are connected
- Locality of connections
    - random
    - local
    - "small world"
    
<table><tr style="background-color: #FFFFFF">
<td><img src="images/l13_s9.2a.png" alt="local connectivity" style="width: 200px;"/></td>
<td><img src="images/l13_s9.2b.png" alt="small world connectivity" style="width: 200px;"/>   </td>
</tr></table>




- Directionality of connectivity
    - feedforward
    - recurrent

<img src="images/l13_sterratt_connectivity.png" alt="feedforward vs recurrent connectivity" style="width: 300px;"/>

### Rate models

As we did earlier, we're going to start with the simplest kind of neuron, one whose probability of spiking is only a function of a single time-varying rate, $r(t)$. This greatly simplifies the math.

In a feedforward network, each neuron's rate is simply a function of its inputs. We can consider the inputs separately, because there's no way for any neuron to affect itself.

By convention, the firing rate of the presynaptic input neurons is represented by the symbol $u$, and the firing rate of the output neurons is represented by $v$. Because each neuron can receive more than one input, $u$ is a vector $\mathbf{u}$. The weights of the synapses from the $\mathbf{u}$ inputs are also a vector, represented as $\mathbf{w}$.

<img src="images/l13_d7.1.png" alt="feedforward model" style="width: 400px;"/>

#### Instantaneous dynamics

We now have to make a decision about time. In an *artificial neural network* (ANN), there either is no time, or it moves in large, discrete steps. Either way, the effect of the inputs on the outputs is instantaneous, so 

$$
v = F(\mathbf{w} \cdot \mathbf{u})
$$

where $F$ is some nonlinear threshold function.

However, we know that voltage and current fluctuations are not instantaneous. It takes time for receptors to open and close, and for the current to propagate from the dendrite to the soma. Therefore, if our goal is to model neural *dynamics*, we need to account for this temporal dependence.

#### Synaptic dynamics

One option is to focus on the dynamics of the synaptic current. If we represent the average synaptic current evoked by a spike as $K_s(t)$, then the total synaptic current is given by convolving the inputs with this function and then performing a weighted sum:

$$
I_s(t) = \sum_{i=1}^N w_i \int_{-\infty}^t K_s(t - \tau) u_i(\tau) d\tau
$$

Note the similarity with a multivariate RF model. However:

- the inputs $u(t)$ are the rates of presynaptic neurons
- we are assuming $K_s(t)$ is the same for all of the inputs

If $K_s(t)$ is modeled as an exponential function with time constant $\tau_s$, then we can rewrite the synaptic current as a differential equation:

$$
\tau_s \frac{\mathrm{d}I_s(t)}{\mathrm{d}t} = - I_s(t) + \mathbf{w} \cdot \mathbf{u}(t)
$$

If the membrane dynamics are relatively fast, we can then model the firing rate as an instantaneous, nonlinear function of the integrated current:

$$
v(t) = F(I_s(t))
$$

#### Membrane dynamics

However, if the time constant of the firing rate is relatively long, then we need to model its dynamics. This is also a simple first-order differential equation: 

$$
\tau_r \frac{\mathrm{d}v(t)}{\mathrm{d}t} = - v(t) + F(I_s(t))
$$

This gives us a system of two differential equations. In practice, however, we can usually do fine with one equation if we assume that the time constant of the firing rate ($\tau_r$) is much longer than the synaptic kernel ($\tau_s$). In this case, the dynamics of the neuron are dominated by the membrane, and the system reduces to:

$$
\tau_r \frac{\mathrm{d}v(t)}{\mathrm{d}t} = - v(t) + F(\mathbf{w} \cdot \mathbf{u}(t))
$$

We will use the membrane-dominated form going forward. However, if instead the synaptic dynamics are much slower than the membrane dynamics, then one can use the system in the previous slide.

### Feedforward network

In a feedforward network, the neurons can be cleanly divided into layers. The simplest network has a single input and a single output layer:

<img src="images/l13_d7.3.png" alt="feedforward network" style="width: 400px;"/>

Each of the $J$ output neurons has the potential to be connected to each of the $I$ input neurons, but the connection weights can vary. Therefore, the structure of the network is described by an $I \times J$ matrix $\mathbf{W}$.

Positive values in $\mathbf{W}$ indicate excitatory connections; negative values indicate inhibitory connections; and zero values indicate the absence of a connection. $\mathbf{W}$ may have many elements set to zero if the connectivity is sparse.

The model for the $j$-th output neuron is specified using the appropriate row of weights from $\mathbf{W}$. For brevity, we will drop the $(t)$ notation for $v$ and $u$. 

$$
\tau_r \frac{\mathrm{d}v_j}{\mathrm{d}t} = - v_j + F\left(\sum_{i} W_{ij} u_i\right)
$$

In vector notation:

$$
\tau_r \frac{\mathrm{d}\mathbf{v}}{\mathrm{d}t} = - \mathbf{v} + F(\mathbf{W} \cdot \mathbf{u})
$$

Note that each neuron depends only on its inputs, so although there are potentially many ODEs in the system, they are **uncoupled**.

#### Associative nets

Hopefully this structure reminds you of the pen-and-paper networks we worked with in our first class.

<img src="images/l13_s9.4.png" alt="associative net" style="width: 700px;"/>

- Each input has the potential to connect to any and all outputs
- Can form associatiations between weak inputs and strong inputs

## Recurrent networks

In a recurrent network, the neurons of the output layer can be connected to each other.

<img src="images/l13_d7.3b.png" alt="feedforward network" style="width: 400px;"/>

The connection weights within the output layer are given by a $J \times J$ matrix $\mathbf{M}$. These currents sum linearly with the currents from the input layer, so the model now looks like:

$$
\tau \frac{\mathrm{d}\mathbf{v}}{\mathrm{d}t} = - \mathbf{v} + F(\mathbf{W} \cdot \mathbf{u} + \mathbf{M} \cdot \mathbf{v})
$$

Because the $\mathbf{W} \cdot \mathbf{u}$ term doesn't depend on the state of the system, it's common to replace it with a single vector $\mathbf{h}$ representing the extrinsic input to each of the $J$ neurons:

$$
\tau \frac{\mathrm{d}\mathbf{v}}{\mathrm{d}t} = - \mathbf{v} + F(\mathbf{h} + \mathbf{M} \cdot \mathbf{v})
$$

In contrast to the feedforward network, the ODEs in a recurrent networks are **coupled** to each other. This opens up a whole realm of computaionally interesting possibilities.


### Linear recurrent networks

Let's start by considering a linear system. That is, with $F(x) = x$, the system of equations becomes:

$$
\tau \frac{\mathrm{d}\mathbf{v}}{\mathrm{d}t} = - \mathbf{v} + \mathbf{h} + \mathbf{M} \cdot \mathbf{v}
$$

#### Eigenvectors and eigenvalues

If the connectivity matrix $\mathbf{M}$ is symmetric (i.e. neuron $i$ is connected to neuron $j$ with the same weight that neuron $j$ is connected to neuron $i$), then we can get an exact solution for $\mathbf{v}$ by orthogonalizing $\mathbf{M}$ into eigenvectors. 

An $N \times N$ array has $N$ eigenvectors that satisfy this equality:

$$
\mathbf{M} \cdot \mathbf{e}_\mu = \lambda_\mu \mathbf{e}_\mu
$$

Eigenvectors of a symmetric matrix are always orthogonal:

$$\mathbf{e}_\mu \cdot \mathbf{e}_\nu = 0 \; \mathrm{for \; all} \; \mu \neq \nu$$

Because of this, any $N$-element vector can be expressed as a weighted sum of the eigenvectors. This includes $\mathbf{v}$:

$$
\mathbf{v}(t) = \sum_\mu^N c_\mu(t) \mathbf{e}_\mu
$$

We can therefore rewrite our model as follows:

\begin{align}
\tau \frac{\mathrm{d}\mathbf{v}}{\mathrm{d}t} & = - \mathbf{v} + \mathbf{M} \cdot \mathbf{v} +\mathbf{h} \\
\tau \sum_\mu^N \frac{\mathrm{d}c_\mu}{\mathrm{d}t}\mathbf{e}_\mu & = - \sum_\mu^N (1 - \lambda_\mu)c_\mu(t)\mathbf{e}_\mu + \mathbf{h}
\end{align}

Then, taking advantage of orthogonality, we can take the dot product on each side with $\mathbf{e}_\nu$ to get a solution for any eigenvector:

$$
\tau \frac{\mathrm{d}c_\nu}{\mathrm{d}t} = - (1 - \lambda_\nu)c_\nu(t) + \mathbf{e}_\nu \cdot \mathbf{h}
$$

The key result here is that the "eigenrate function" $c_\nu(t)$ depends only on itself and on a weighted sum of the extrinsic inputs, $\mathbf{e}_\nu \cdot \mathbf{h}$. In other words, the system reduces to a set of **uncoupled**, first-order ODEs.

#### Selective amplification

A useful consequence of being able to orthogonalize the connectivity matrix is that the rate of each eigenvector has a simple, analytical solution:

$$
c_\nu(t) = \frac{\mathbf{e}_\nu \cdot \mathbf{h}}{1 - \lambda_\nu}
\left(1 - \exp \left(- \frac{t(1 - \lambda_\nu)}{\tau}\right)\right) + 
c_\nu(0) \exp \left(- \frac{t(1 - \lambda_\nu)}{\tau}\right)
$$

The eigenvalues here have a similar interpretation as before:

- $\lambda_\nu > 1$: the system will diverge exponentially to infinity
- $\lambda_\nu < 1$: $c_\nu(t)$ will exponentially converge to the steady-state value $(\mathbf{e}_\nu \cdot \mathbf{h})/(1 - \lambda_\nu)$

Moreover, the steady state of the entire network is a weighted sum of the eigenstates:

$$
\mathbf{v}_\infty = \sum_\nu^N \frac{\mathbf{e}_\nu \cdot \mathbf{h}}{1 - \lambda_\nu} \mathbf{e}_\nu
$$

If one of the eigenvalues ($\lambda_1$) is close to 1, and all the others are relatively small, then the denominator $(1 - \lambda_1)$ will be close to zero, and the projection of the stimulus onto this eigenvector will dominate the response:

$$
\mathbf{v}_\infty \approx \frac{\mathbf{e}_1 \cdot \mathbf{h}}{1 - \lambda_1} \mathbf{e}_1
$$

For example, this figure from Carandini and Ringach (1997) compares the input and output of a linear recurrent network modeling visual orientation selectivity.

<img src="images/l13_d7.8.png" alt="selective amplification" style="width: 400px;"/>

The input is relatively noisy and untuned, but the network's eigenvector with the largest eigenvalue ($\lambda_1 = 0.9$) has a broad peak at around 0 degrees, and this selectively amplifies the projection of the input onto that eigenvector.

#### Integrator networks

When $\lambda_1$ is exactly equal to 1.0, the network becomes a linear integrator.

\begin{align}
\tau \frac{\mathrm{d}c_\nu}{\mathrm{d}t} & = - (1 - \lambda_\mu)c_\mu(t) + \mathbf{e}_\nu \cdot \mathbf{h} \\
\tau \frac{\mathrm{d} c_1}{\mathrm{d}t} & = \mathbf{e}_1 \cdot \mathbf{h}
\end{align}

This means that activity in the network will persist in the absence of an input. This kind of network behavior is seen in brain areas responsible for maintaining eye position. For example, from Seung et al (2000):

<img src="images/l13_d7.7.png" alt="linear integrator" style="width: 400px;"/>

### Nonlinear recurrent networks

One of the simplest nonlinearities we can add is rectification, so that firing rates cannot be less than zero. 

$$
F(\cdot) = [\cdot]_+
$$

This model also exhibits selective amplification, but with an even sharper peak:

<img src="images/l13_d7.9.png" alt="selective amplification - nonlinear" style="width: 400px;"/>

Even more interestingly, the model exhibits winner-take-all dynamics. When the input has activity around two conflicting directions, the network response is only at the higher peak:

<img src="images/l13_d7.12.png" alt="selective amplification - winner-take-all" style="width: 400px;"/>

## Excitatory-Inhibitory Networks

One problem with assuming that the connectivity matrix $\mathbf{M}$ is symmetric is that it violates Dale's Law, which states that neurons are either excitatory or inhibitory. So another change that is often made to network models is to split out the neurons into groups.

\begin{align}
\tau_E \frac{\mathrm{d}\mathbf{v}_E}{\mathrm{d}t} & = - \mathbf{v}_E + F_E(\mathbf{h}_E + \mathbf{M}_{EE} \cdot \mathbf{v}_E + \mathbf{M}_{EI} \cdot \mathbf{v}_I ) \\
\tau_I \frac{\mathrm{d}\mathbf{v}_I}{\mathrm{d}t} & = - \mathbf{v}_I + F_I(\mathbf{h}_I + \mathbf{M}_{IE} \cdot \mathbf{v}_E + \mathbf{M}_{II} \cdot \mathbf{v}_I )
\end{align}

Note that the excitatory (glutamatergic) and inhibitory (GABAergic) neurons have different time constants, activation functions, and inputs. Further splitting of cell types is possible; for example, the inhibitory neurons can be split into separate populations of parvalbumin- and somatostatin-positive cells.

### Homogeneous networks

If we assume that the excitatory and inhibitory populations are homogeneous, with the same rates across all the cells, we can see some of the interesting dynamics that become possible with more complex connectivity and nonlinearity.

We will use the nonlinear rectification function $F(\cdot) = [\cdot]_+$

\begin{align}
\tau_E \frac{\mathrm{d}v_E}{\mathrm{d}t} & = - v_E + [h_E + M_{EE} v_E + M_{EI} v_I - \gamma_E]_+ \\
\tau_I \frac{\mathrm{d}v_I}{\mathrm{d}t} & = - v_I + [h_I + M_{IE} v_E + M_{II} v_I - \gamma_I]_+
\end{align}



Because this is a two-dimensional system, we can use the geometric analysis methods we discussed earlier for single neuron models.

The nullclines are straight lines that intersect at a single point. The stability of this fixed point depends on the sign of the eigenvalues of the Jacobian. For some parameter values, the fixed point is a stable focus:

<img src="images/l13_d7.18.png" alt="homogeneous - stable" style="width: 400px;"/>

For other parameter values, the fixed point is an unstable focus:

<img src="images/l13_d7.19.png" alt="homogeneous - stable" style="width: 400px;"/>

Why is the focus surrounded by a stable limit cycle?